In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from collections import Counter
from typing import List, Dict, Tuple, Optional

# NLP libraries
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# ============================================================================
# PROJECT SETUP & MODEL INITIALIZATION
# ============================================================================

PROJECT_DIR = Path().resolve()
INTERIM_DIR = PROJECT_DIR / "data/interim"
FILE_PATH = PROJECT_DIR / "data/raw/lamoda_reviews.csv"

# Проверяем, какие модели есть в кеше
import os
from pathlib import Path as PathLib

cache_dir = PathLib.home() / ".cache" / "huggingface" / "hub"
print("Checking local cache for models...")
print(f"Cache directory: {cache_dir}")

# Список моделей для проверки
model_candidates = [
    'all-MiniLM-L6-v2',
    'all-mpnet-base-v2', 
    'paraphrase-multilingual-MiniLM-L12-v2',
    'distilbert-base-nli-mean-tokens',
    'paraphrase-MiniLM-L6-v2',
]

sentence_model = None
loaded_model_name = None
USE_TFIDF_FALLBACK = False

# Пробуем загрузить из кеша
for model_name in model_candidates:
    try:
        print(f"\nTrying model: {model_name}...")
        sentence_model = SentenceTransformer(model_name, local_files_only=True)
        loaded_model_name = model_name
        print(f"✓ Model '{model_name}' loaded successfully from cache!")
        break
    except Exception as e:
        print(f"✗ {model_name}: Not found in cache")
        continue

# Если ничего не найдено, используем TF-IDF как fallback
if sentence_model is None:
    print("\n" + "="*80)
    print("⚠ No sentence transformer models found in cache")
    print("Using TF-IDF as fallback for embeddings")
    print("="*80)
    USE_TFIDF_FALLBACK = True
else:
    print(f"\n{'='*80}")
    print(f"✓ Using model: {loaded_model_name}")
    print(f"  Embedding dimension: {sentence_model.get_sentence_embedding_dimension()}")
    print("="*80)


No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.
No sentence-transformers model found with name sentence-transformers/all-mpnet-base-v2. Creating a new one with mean pooling.
No sentence-transformers model found with name sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2. Creating a new one with mean pooling.
No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


Loading sentence transformer model...
Trying different models...

Trying model: all-MiniLM-L6-v2...
✗ all-MiniLM-L6-v2: OSError

Trying model: all-mpnet-base-v2...
✗ all-mpnet-base-v2: OSError

Trying model: paraphrase-multilingual-MiniLM-L12-v2...
✗ paraphrase-multilingual-MiniLM-L12-v2: OSError

Trying model: sentence-transformers/all-MiniLM-L6-v2...
✗ sentence-transformers/all-MiniLM-L6-v2: OSError

No models found in cache. Trying to download 'all-MiniLM-L6-v2'...


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1017)')))"), '(Request ID: a02a200d-5308-477a-9455-83a29fd6fdc0)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1017)')))"), '(Request ID: 3edc9ac1-613b-4fde-ad61-b85f32816f23)')' thrown while requesting HEAD https://huggingface.co/sente

KeyboardInterrupt: 

In [ ]:
# ============================================================================
# LOAD DATA
# ============================================================================

print("Loading data...")
df = pd.read_csv(FILE_PATH)
print(f"Loaded {len(df)} reviews")
print(f"Unique products: {df['product_sku'].nunique()}")
print(f"Categories: {df['good_type'].unique()}")
print(f"Subtypes: {df['good_subtype'].nunique()} unique subtypes")
df.head()


In [ ]:
# ============================================================================
# (1) TEXT CLEANING - МЯГКИЕ ПРАВИЛА
# ============================================================================

def clean_text_soft(text: str) -> str:
    """
    Мягкая очистка текста: минимальная обработка
    - Удаляем множественные пробелы
    - Сохраняем все знаки препинания и эмодзи
    - Приводим к нижнему регистру
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    # Удаляем множественные пробелы и переносы строк
    text = re.sub(r'\s+', ' ', text)
    # Удаляем только управляющие символы, сохраняем все остальное
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return text.strip().lower()


# Применяем очистку
print("Cleaning texts...")
df['clean_text'] = df['comment_text'].apply(clean_text_soft)
df['clean_text'] = df['clean_text'].fillna('')

# Удаляем пустые отзывы
df = df[df['clean_text'].str.len() > 10].copy()
print(f"After cleaning: {len(df)} reviews")
df[['comment_text', 'clean_text', 'good_type', 'good_subtype']].head()

,comment_id,product_sku,comment_text,name,good_type,good_subtype
0,395865747,MP002XB078CD,"Купили сыну на день рождения!Теплая, но не жар...",Куртка утепленная,Clothes,OUTWEAR
1,436891792,MP002XB07Z8I,"Приятная вещь, симпатично смотрится",Свитшот,Clothes,SWEATSHIRTS
2,383386833,MP002XC00LSY,Произведено в Турции. Качество хорошее и разме...,Поло,Clothes,TEE-SHIRTS & POLOS
3,400670943,MP002XC01NLE,Классный комплект,Боди и ползунки,Clothes,TEE-SHIRTS & POLOS
4,388822372,MP002XG03J2N,"Отличные перчатки, плотные, яркие. Ребенку 8 л...",Перчатки,Accs,GLOVES & MITTENS


In [ ]:
# ============================================================================
# (2) EMBEDDINGS - ОДНИ И ТЕ ЖЕ ДЛЯ ВСЕХ ОТЗЫВОВ
# ============================================================================

def get_embeddings(texts: List[str], batch_size: int = 32) -> np.ndarray:
    """
    Получение embeddings для списка текстов
    Используется одна и та же модель для всех отзывов
    Если sentence transformer недоступен, используем TF-IDF
    """
    if USE_TFIDF_FALLBACK:
        print(f"Generating TF-IDF embeddings for {len(texts)} texts...")
        vectorizer = TfidfVectorizer(
            max_features=384,  # Примерно как у MiniLM-L6-v2
            ngram_range=(1, 2),
            min_df=1,
            max_df=0.95
        )
        embeddings = vectorizer.fit_transform(texts).toarray()
        print(f"✓ Generated TF-IDF embeddings shape: {embeddings.shape}")
        return embeddings
    else:
        if sentence_model is None:
            raise ValueError("Sentence transformer model is not loaded!")
        
        print(f"Generating sentence transformer embeddings for {len(texts)} texts...")
        embeddings = sentence_model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        print(f"✓ Generated embeddings shape: {embeddings.shape}")
        return embeddings


# Генерируем embeddings для всех отзывов
print("="*80)
print("STEP 2: GENERATING EMBEDDINGS")
print("="*80)

# Для тестирования можно использовать подмножество
USE_SUBSET = True
if USE_SUBSET:
    df_sample = df.head(10000).copy()  # Используем 10k для тестирования
    print(f"Using subset: {len(df_sample)} reviews")
else:
    df_sample = df.copy()

# Получаем embeddings
review_texts = df_sample['clean_text'].tolist()
embeddings = get_embeddings(review_texts)

# Сохраняем embeddings в DataFrame
df_sample['embedding'] = list(embeddings)
print(f"✓ Embeddings added to dataframe")

In [ ]:
# ============================================================================
# (3) CLUSTERING - ОДНА ЛОГИКА КЛАСТЕРИЗАЦИИ
# ============================================================================

def cluster_embeddings(
    embeddings: np.ndarray,
    n_clusters: Optional[int] = None,
    min_clusters: int = 3,
    max_clusters: int = 10
) -> Tuple[np.ndarray, int]:
    """
    Кластеризация embeddings с автоматическим выбором числа кластеров
    Используется одна и та же логика для всех продуктов
    """
    n_samples = len(embeddings)
    
    # Автоматический выбор числа кластеров
    if n_clusters is None:
        # Используем правило локтя или фиксированный диапазон
        if n_samples < 10:
            n_clusters = min(3, n_samples)
        elif n_samples < 50:
            n_clusters = min(5, n_samples // 2)
        else:
            # Пробуем несколько значений и выбираем лучшее по silhouette score
            best_score = -1
            best_k = min_clusters
            
            for k in range(min_clusters, min(max_clusters + 1, n_samples // 2)):
                try:
                    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
                    labels = kmeans.fit_predict(embeddings)
                    score = silhouette_score(embeddings, labels)
                    if score > best_score:
                        best_score = score
                        best_k = k
                except:
                    continue
            
            n_clusters = best_k
    
    # Выполняем кластеризацию
    print(f"Clustering {n_samples} embeddings into {n_clusters} clusters...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(embeddings)
    
    print(f"✓ Clustering completed. Cluster distribution:")
    unique, counts = np.unique(cluster_labels, return_counts=True)
    for cluster_id, count in zip(unique, counts):
        print(f"  Cluster {cluster_id}: {count} reviews")
    
    return cluster_labels, n_clusters


# Применяем кластеризацию на уровне продукта
print("="*80)
print("STEP 3: CLUSTERING EMBEDDINGS")
print("="*80)

# Группируем по продуктам и кластеризуем отзывы каждого продукта
results = []

for sku, group in df_sample.groupby('product_sku'):
    if len(group) < 3:  # Минимум 3 отзыва для кластеризации
        continue
    
    # Получаем embeddings для отзывов этого продукта
    product_embeddings = np.array(group['embedding'].tolist())
    
    # Кластеризуем
    cluster_labels, n_clusters = cluster_embeddings(product_embeddings)
    
    # Добавляем метки кластеров
    group = group.copy()
    group['cluster'] = cluster_labels
    
    results.append(group)

# Объединяем результаты
df_clustered = pd.concat(results, ignore_index=True)
print(f"\n✓ Clustered {len(df_clustered)} reviews across {df_clustered['product_sku'].nunique()} products")
df_clustered[['product_sku', 'clean_text', 'cluster', 'good_type', 'good_subtype']].head(10)


In [ ]:
# ============================================================================
# (4) ASPECT EXTRACTION - CATEGORY-AWARE
# ============================================================================

# Словари аспектов для разных категорий товаров
CATEGORY_ASPECTS = {
    'Clothes': {
        'размер': ['размер', 'размеру', 'размера', 'размеры', 'подошёл', 'подошла', 'подошли', 'маленький', 'большой'],
        'материал': ['материал', 'ткань', 'хлопок', 'синтетика', 'шерсть', 'полиэстер'],
        'качество': ['качество', 'качественный', 'качественная', 'качественные', 'хороший', 'плохой'],
        'цвет': ['цвет', 'цвета', 'окраска', 'яркий', 'яркая', 'яркие', 'красивый'],
        'комфорт': ['удобный', 'удобная', 'удобные', 'комфортный', 'комфортная', 'комфортно'],
        'стиль': ['стильный', 'стильная', 'стильные', 'модный', 'модная', 'модные'],
        'цена': ['цена', 'стоимость', 'дешево', 'дорого', 'дорогой', 'дешевый']
    },
    'Shoes': {
        'размер': ['размер', 'размеру', 'размера', 'размеры', 'подошёл', 'подошла', 'подошли'],
        'комфорт': ['удобный', 'удобная', 'удобные', 'комфортный', 'комфортная', 'комфортно', 'натирает'],
        'качество': ['качество', 'качественный', 'качественная', 'качественные', 'прочный', 'прочная'],
        'материал': ['кожа', 'кожаный', 'кожаная', 'текстиль', 'резина'],
        'внешний вид': ['красивый', 'красивая', 'красивые', 'симпатичный', 'стильный'],
        'цена': ['цена', 'стоимость', 'дешево', 'дорого']
    },
    'Accs': {
        'качество': ['качество', 'качественный', 'качественная', 'качественные'],
        'материал': ['материал', 'кожа', 'металл', 'пластик'],
        'внешний вид': ['красивый', 'красивая', 'красивые', 'симпатичный', 'стильный'],
        'размер': ['размер', 'размеру', 'размера', 'размеры'],
        'цена': ['цена', 'стоимость', 'дешево', 'дорого']
    }
}

# Общие аспекты для всех категорий
COMMON_ASPECTS = {
    'качество': ['качество', 'качественный', 'качественная', 'качественные', 'хороший', 'плохой'],
    'цена': ['цена', 'стоимость', 'дешево', 'дорого', 'дорогой', 'дешевый'],
    'внешний вид': ['красивый', 'красивая', 'красивые', 'симпатичный', 'симпатично', 'смотрится']
}


def extract_aspects_from_cluster(
    cluster_texts: List[str],
    cluster_embeddings: np.ndarray,
    category: str,
    good_subtype: Optional[str] = None
) -> Dict[str, str]:
    """
    Извлечение аспектов из кластера с учетом категории товара
    
    Args:
        cluster_texts: Тексты отзывов в кластере
        cluster_embeddings: Embeddings отзывов в кластере
        category: Категория товара (Clothes, Shoes, Accs)
        good_subtype: Подтип товара (опционально)
    
    Returns:
        Словарь {aspect_name: representative_text}
    """
    if len(cluster_texts) == 0:
        return {}
    
    # Выбираем словарь аспектов для категории
    aspect_keywords = CATEGORY_ASPECTS.get(category, COMMON_ASPECTS)
    
    # Находим наиболее репрезентативный отзыв в кластере (ближайший к центру)
    center = cluster_embeddings.mean(axis=0)
    distances = np.linalg.norm(cluster_embeddings - center, axis=1)
    representative_idx = distances.argmin()
    representative_text = cluster_texts[representative_idx]
    
    # Определяем, какие аспекты упоминаются в кластере
    detected_aspects = {}
    cluster_text_lower = ' '.join(cluster_texts).lower()
    
    for aspect_name, keywords in aspect_keywords.items():
        # Проверяем наличие ключевых слов
        if any(keyword in cluster_text_lower for keyword in keywords):
            # Используем репрезентативный текст как значение аспекта
            detected_aspects[aspect_name] = representative_text[:100]  # Ограничиваем длину
    
    return detected_aspects


def extract_aspects_for_product(
    product_df: pd.DataFrame,
    category: str,
    good_subtype: Optional[str] = None
) -> Dict[str, List[str]]:
    """
    Извлечение аспектов для продукта на основе кластеров
    
    Returns:
        Словарь {aspect_name: [representative_texts]}
    """
    product_aspects = {}
    
    # Обрабатываем каждый кластер отдельно
    for cluster_id in product_df['cluster'].unique():
        cluster_df = product_df[product_df['cluster'] == cluster_id]
        cluster_texts = cluster_df['clean_text'].tolist()
        cluster_embeddings = np.array(cluster_df['embedding'].tolist())
        
        # Извлекаем аспекты из кластера
        cluster_aspects = extract_aspects_from_cluster(
            cluster_texts,
            cluster_embeddings,
            category,
            good_subtype
        )
        
        # Объединяем аспекты
        for aspect_name, text in cluster_aspects.items():
            if aspect_name not in product_aspects:
                product_aspects[aspect_name] = []
            product_aspects[aspect_name].append(text)
    
    return product_aspects


In [ ]:
# Применяем извлечение аспектов для всех продуктов
print("="*80)
print("STEP 4: ASPECT EXTRACTION (CATEGORY-AWARE)")
print("="*80)

product_results = []

for sku, product_df in df_clustered.groupby('product_sku'):
    category = product_df['good_type'].iloc[0] if 'good_type' in product_df.columns else None
    good_subtype = product_df['good_subtype'].iloc[0] if 'good_subtype' in product_df.columns else None
    product_name = product_df['name'].iloc[0] if 'name' in product_df.columns else None
    
    # Извлекаем аспекты
    aspects = extract_aspects_for_product(product_df, category, good_subtype)
    
    # Формируем теги из аспектов (берем по одному репрезентативному тексту на аспект)
    tags = []
    for aspect_name, texts in aspects.items():
        if texts:
            # Берем первый (наиболее репрезентативный) текст
            tag = texts[0]
            # Укорачиваем если нужно
            if len(tag) > 80:
                tag = tag[:77] + "..."
            tags.append(tag)
    
    product_results.append({
        'product_sku': sku,
        'product_name': product_name,
        'category': category,
        'good_subtype': good_subtype,
        'num_reviews': len(product_df),
        'num_clusters': product_df['cluster'].nunique(),
        'aspects': aspects,
        'tags': tags[:6],  # Максимум 6 тегов
        'tags_count': min(len(tags), 6)
    })

results_df = pd.DataFrame(product_results)
print(f"\n✓ Extracted aspects for {len(results_df)} products")
print(f"Average tags per product: {results_df['tags_count'].mean():.2f}")
results_df.head(10)


In [ ]:
# ============================================================================
# EXAMINE RESULTS
# ============================================================================

print("="*80)
print("RESULTS EXAMINATION")
print("="*80)

# Показываем примеры для разных категорий
for category in results_df['category'].unique()[:3]:
    if pd.isna(category):
        continue
    
    cat_products = results_df[results_df['category'] == category].head(3)
    print(f"\n{'='*80}")
    print(f"Category: {category}")
    print("="*80)
    
    for _, row in cat_products.iterrows():
        print(f"\nProduct: {row['product_name']} (SKU: {row['product_sku']})")
        print(f"Reviews: {row['num_reviews']}, Clusters: {row['num_clusters']}")
        print(f"Tags ({row['tags_count']}):")
        for i, tag in enumerate(row['tags'], 1):
            print(f"  {i}. {tag}")
        
        # Показываем обнаруженные аспекты
        if row['aspects']:
            print(f"\nDetected aspects:")
            for aspect_name, texts in row['aspects'].items():
                print(f"  - {aspect_name}: {len(texts)} mentions")


In [ ]:
# ============================================================================
# EXPORT RESULTS
# ============================================================================

import json

# Сохраняем результаты
output_path = INTERIM_DIR / "product_aspects.csv"

# Подготавливаем данные для сохранения
export_df = results_df.copy()
export_df['tags'] = export_df['tags'].apply(lambda x: ' | '.join(x) if isinstance(x, list) else '')
export_df['aspects'] = export_df['aspects'].apply(lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, dict) else '{}')

export_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"✓ Results saved to: {output_path}")

# Также сохраняем в JSON
output_json_path = INTERIM_DIR / "product_aspects.json"
results_df.to_json(output_json_path, orient='records', force_ascii=False, indent=2)
print(f"✓ Results also saved to: {output_json_path}")

# Статистика
print("\n" + "="*80)
print("STATISTICS")
print("="*80)
print(f"Total products processed: {len(results_df)}")
print(f"Products with tags: {(results_df['tags_count'] > 0).sum()}")
print(f"Average tags per product: {results_df['tags_count'].mean():.2f}")
print(f"Average clusters per product: {results_df['num_clusters'].mean():.2f}")
print(f"\nTags distribution:")
print(results_df['tags_count'].value_counts().sort_index())
print(f"\nCategories distribution:")
print(results_df['category'].value_counts())


In [ ]:
# ============================================================================
# PIPELINE SUMMARY
# ============================================================================

print("="*80)
print("PIPELINE SUMMARY")
print("="*80)
print("""
Pipeline structure:
  1. Отзывы + category + product_type
     ↓
  2. Очистка (мягкие правила) - минимальная обработка текста
     ↓
  3. Embeddings (одни и те же) - единая модель для всех отзывов
     ↓
  4. Кластеризация (одна логика) - единая логика кластеризации
     ↓
  5. Aspect extraction ← category-aware - извлечение с учетом категории

Key features:
  - Мягкая очистка: сохраняем максимум информации
  - Единые embeddings: одна модель для всех отзывов
  - Автоматическая кластеризация: выбор оптимального числа кластеров
  - Category-aware aspects: разные словари аспектов для разных категорий
""")


In [ ]:
# ============================================================================
# PROCESS FULL DATASET (OR SAMPLE)
# ============================================================================

# Обрабатываем датасет
# Для быстрого тестирования используем sample_size
# Для полной обработки установите sample_size=None

SAMPLE_SIZE = 50  # Обработать топ-50 товаров по количеству отзывов
# SAMPLE_SIZE = None  # Раскомментируйте для обработки всех товаров

print("Starting batch processing...")
results_df = process_dataset(df, sample_size=SAMPLE_SIZE)

print(f"\nProcessed {len(results_df)} products")
print(f"Average tags per product: {results_df['tags_count'].mean():.2f}")
print(f"\nResults preview:")
results_df.head(10)


In [ ]:
# ============================================================================
# EXPORT RESULTS
# ============================================================================

# Сохраняем результаты
output_path = INTERIM_DIR / "product_tags.csv"
results_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Results saved to: {output_path}")

# Также сохраняем в более читаемом формате (JSON)
import json
output_json_path = INTERIM_DIR / "product_tags.json"

# Конвертируем список тегов в строку для JSON
json_data = results_df.copy()
json_data['tags'] = json_data['tags'].apply(lambda x: json.dumps(x, ensure_ascii=False))

json_data.to_json(output_json_path, orient='records', force_ascii=False, indent=2)
print(f"Results also saved to: {output_json_path}")

# Показываем статистику
print("\n" + "="*80)
print("STATISTICS")
print("="*80)
print(f"Total products processed: {len(results_df)}")
print(f"Products with tags: {(results_df['tags_count'] > 0).sum()}")
print(f"Average tags per product: {results_df['tags_count'].mean():.2f}")
print(f"Min tags: {results_df['tags_count'].min()}")
print(f"Max tags: {results_df['tags_count'].max()}")
print(f"\nTags distribution:")
print(results_df['tags_count'].value_counts().sort_index())


In [ ]:
# ============================================================================
# VISUALIZATION: EXAMINE RESULTS
# ============================================================================

# Показываем примеры результатов для разных категорий
print("Examples by category:\n")

for category in results_df['category'].unique()[:3]:
    if pd.isna(category):
        continue
    cat_products = results_df[results_df['category'] == category].head(3)
    print(f"\n{category}:")
    print("-" * 80)
    for _, row in cat_products.iterrows():
        print(f"\n  SKU: {row['product_sku']}")
        print(f"  Name: {row['product_name']}")
        print(f"  Reviews: {row['num_reviews']}")
        print(f"  Tags ({row['tags_count']}):")
        for tag in row['tags']:
            print(f"    - {tag}")


(1774267, 6)